In [ ]:
#Your final output should provide a visualized summary of the sentiments expressed in Tweets sent 
#out by the following news organizations: BBC, CBS, CNN, Fox, and New York times.

In [1]:
#import dependencies
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#vader sentimenet
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Twitter API Keys
from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [30]:
target_news = ('@BBC', '@CBS', '@CNN', '@FoxNews', '@nytimes')

data = {}
  
# Variables for holding sentiments
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
    
# Variable for max_id
oldest_tweet = None

for news in target_news:

    # Get all tweets from home feed
    public_tweets = api.user_timeline(news, max_id = oldest_tweet)

    for tweet in public_tweets:
        results = analyzer.polarity_scores(tweet['text'])
        compound_list.append(results['compound'])
        positive_list.append(results['pos'])
        neutral_list.append(results['neu'])
        negative_list.append(results['neg'])

        # Get Tweet ID, subtract 1, and assign to oldest_tweet
        oldest_tweet = tweet['id'] - 1
    
    data_set = {'compound': compound_list, 'positive': positive_list,
               'neutral': neutral_list, 'negative': negative_list}
    
    data[news] = data_set
    

In [29]:
CBS = pd.DataFrame(data['@CBS'])
BBC = pd.DataFrame(data['@BBC'])
FOX = cbs = pd.DataFrame(data['@Fox'])
print(cbs.shape)

(100, 4)


In [ ]:
#The first plot will be and/or feature the following:

#Be a scatter plot of sentiments of the last 100 tweets sent out by each news organization,
#ranging from -1.0 to 1.0, where a score of 0 expresses a neutral sentiment, -1 the most negative 
#sentiment possible, and +1 the most positive sentiment possible.

#Each plot point will reflect the compound sentiment of a tweet.

#Sort each plot point by its relative timestamp.

In [ ]:
#The second plot will be a bar plot visualizing the overall sentiments of the last 100 tweets from each 
#organization. For this plot, you will again aggregate the compound sentiments analyzed by VADER.